In [2]:
"""
Skeleton code showing how to load and run the TensorFlow SavedModel export package from Lobe.
"""
import argparse
import os
import json
import numpy as np
import sys 
from threading import Lock
from pprint import pprint 
from PIL import Image 
from tensorflow.keras import Model, models, layers, regularizers, preprocessing, datasets, metrics, losses, optimizers
from tqdm import tqdm
from sklearn.metrics import classification_report

# printing only warnings and error messages
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

try:
    import tensorflow as tf
    from PIL import Image
except ImportError:
    raise ImportError("ERROR: Failed to import libraries. Please refer to READEME.md file\n")

EXPORT_MODEL_VERSION = 1

base_path = os.path.abspath('/tf/eipi10/jian-xu3/qbz95')
sys.path.append(base_path)

import qbz95
from qbz95 import tf as qtf


# set gpu memory
qtf.utils.set_gpu_memory_growth()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
class TFModel:
    def __init__(self, dir_path, classes) -> None:
        # Assume model is in the parent directory for this file
        self.model_dir = dir_path
        # make sure our exported SavedModel folder exists
        with open(os.path.join(self.model_dir, "signature.json"), "r") as f:
            self.signature = json.load(f)
        self.model_file = os.path.join(self.model_dir, self.signature.get("filename"))
        if not os.path.isfile(self.model_file):
            raise FileNotFoundError(f"Model file does not exist")
        self.inputs = self.signature.get("inputs")
        self.outputs = self.signature.get("outputs")
        self.lock = Lock()

        # loading the saved model
        self.model = tf.saved_model.load(tags=self.signature.get("tags"), export_dir=self.model_dir)
        self.predict_fn = self.model.signatures["serving_default"]

        # Look for the version in signature file.
        # If it's not found or the doesn't match expected, print a message
        version = self.signature.get("export_model_version")
        if version is None or version != EXPORT_MODEL_VERSION:
            print(
                f"There has been a change to the model format. Please use a model with a signature 'export_model_version' that matches {EXPORT_MODEL_VERSION}."
            )
            
        self.classes = classes

    def predict(self, image: Image.Image) -> dict:
        # pre-processing the image before passing to model
        image = self.process_image(image, self.inputs.get("Image").get("shape"))

        with self.lock:
            # create the feed dictionary that is the input to the model
            feed_dict = {}
            # first, add our image to the dictionary (comes from our signature.json file)
            feed_dict[list(self.inputs.keys())[0]] = tf.convert_to_tensor(image)
            # run the model!
            outputs = self.predict_fn(**feed_dict)
            # return the processed output
            return self.process_output(outputs)
        
    def predict_from_arrays(self, image_arrays):
        predictions = []
        for image_array in tqdm(image_arrays):
            image = Image.fromarray(image_array)
            outputs = self.predict(image)
            prediction = outputs['predictions'][0]['label']
            predictions.append(self.classes.index(prediction))
        return np.array(predictions)

    def process_image(self, image, input_shape) -> np.ndarray:
        """
        Given a PIL Image, center square crop and resize to fit the expected model input, and convert from [0,255] to [0,1] values.
        """
        width, height = image.size
        # ensure image type is compatible with model and convert if not
        if image.mode != "RGB":
            image = image.convert("RGB")
        # center crop image (you can substitute any other method to make a square image, such as just resizing or padding edges with 0)
        if width != height:
            square_size = min(width, height)
            left = (width - square_size) / 2
            top = (height - square_size) / 2
            right = (width + square_size) / 2
            bottom = (height + square_size) / 2
            # Crop the center of the image
            image = image.crop((left, top, right, bottom))
        # now the image is square, resize it to be the right shape for the model input
        input_width, input_height = input_shape[1:3]
        if image.width != input_width or image.height != input_height:
            image = image.resize((input_width, input_height))

        # make 0-1 float instead of 0-255 int (that PIL Image loads by default)
        image = np.asarray(image) / 255.0
        # pad with an extra batch dimension as expected by the model
        return np.expand_dims(image, axis=0).astype(np.float32)

    def process_output(self, outputs) -> dict:
        # do a bit of postprocessing
        out_keys = ["label", "confidence"]
        results = {}
        # since we actually ran on a batch of size 1, index out the items from the returned numpy arrays
        for key, tf_val in outputs.items():
            val = tf_val.numpy().tolist()[0]
            if isinstance(val, bytes):
                val = val.decode()
            results[key] = val
        confs = results["Confidences"]
        labels = self.signature.get("classes").get("Label")
        output = [dict(zip(out_keys, group)) for group in zip(labels, confs)]
        sorted_output = {"predictions": sorted(output, key=lambda k: k["confidence"], reverse=True)}
        return sorted_output




In [4]:
classes =  ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
dir_path = "/tf/eipi10/jian-xu3/adp_claim_image/output/cifar10_50_TensorFlow"
model = TFModel(dir_path=dir_path, classes=classes) 

image_path = "/tf/eipi10/xuxiangwen.github.io/_notes/05-ai/54-tensorflow/image_classification/output/cifar10/data/cifar10_test/bird/1047.png"
image = Image.open(image_path)

outputs = model.predict(image)
pprint(f"Predicted: {outputs}") 


("Predicted: {'predictions': [{'label': 'bird', 'confidence': "
 "0.9999884366989136}, {'label': 'cat', 'confidence': 6.133375336503377e-06}, "
 "{'label': 'deer', 'confidence': 4.627039288607193e-06}, {'label': 'frog', "
 "'confidence': 7.819590450708347e-07}, {'label': 'dog', 'confidence': "
 "2.5593249741717727e-10}, {'label': 'car', 'confidence': "
 "2.1317153771349684e-12}, {'label': 'plane', 'confidence': "
 "3.66488862369764e-14}, {'label': 'horse', 'confidence': "
 "6.278238274718502e-15}, {'label': 'truck', 'confidence': "
 "4.8913437963339895e-15}, {'label': 'ship', 'confidence': "
 '4.249912043104342e-17}]}')


In [6]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [8]:
train_image_counts = [1000, 500, 200, 100, 50]
for train_image_count in train_image_counts:
    print('-'*25, train_image_count, '-'*25)
    dir_path = f"/tf/eipi10/jian-xu3/adp_claim_image/output/cifar10_{train_image_count}_TensorFlow"
    model = TFModel(dir_path=dir_path, classes=classes) 
    prediction_labels = model.predict_from_arrays(test_images)
    print(classification_report(test_labels, prediction_labels, digits=4, target_names=classes))

------------------------- 1000 -------------------------


100%|██████████| 10000/10000 [02:02<00:00, 81.67it/s]


              precision    recall  f1-score   support

       plane     0.8270    0.8510    0.8388      1000
         car     0.9654    0.5860    0.7293      1000
        bird     0.9031    0.6620    0.7640      1000
         cat     0.7894    0.4010    0.5318      1000
        deer     0.7905    0.5850    0.6724      1000
         dog     0.7722    0.6340    0.6963      1000
        frog     0.7125    0.8920    0.7922      1000
       horse     0.4866    0.9660    0.6472      1000
        ship     0.8743    0.9110    0.8923      1000
       truck     0.7132    0.9150    0.8016      1000

    accuracy                         0.7403     10000
   macro avg     0.7834    0.7403    0.7366     10000
weighted avg     0.7834    0.7403    0.7366     10000

------------------------- 500 -------------------------


100%|██████████| 10000/10000 [02:03<00:00, 80.98it/s]


              precision    recall  f1-score   support

       plane     0.7698    0.8460    0.8061      1000
         car     0.9262    0.8540    0.8887      1000
        bird     0.8874    0.6700    0.7635      1000
         cat     0.7733    0.4230    0.5469      1000
        deer     0.8296    0.6720    0.7425      1000
         dog     0.6371    0.8620    0.7327      1000
        frog     0.8601    0.7990    0.8284      1000
       horse     0.6130    0.9220    0.7364      1000
        ship     0.9302    0.7730    0.8443      1000
       truck     0.7456    0.9320    0.8284      1000

    accuracy                         0.7753     10000
   macro avg     0.7972    0.7753    0.7718     10000
weighted avg     0.7972    0.7753    0.7718     10000

------------------------- 200 -------------------------


...

              precision    recall  f1-score   support

       plane     0.8121    0.7910    0.8014      1000
         car     0.9416    0.8540    0.8956      1000
        bird     0.7912    0.7920    0.7916      1000
         cat     0.6887    0.6880    0.6883      1000
        deer     0.8074    0.5700    0.6682      1000
         dog     0.7188    0.8640    0.7847      1000
        frog     0.7764    0.8470    0.8101      1000
       horse     0.8773    0.7010    0.7793      1000
        ship     0.6923    0.9720    0.8087      1000
       truck     0.9160    0.8400    0.8764      1000

    accuracy                         0.7919     10000
   macro avg     0.8022    0.7919    0.7904     10000
weighted avg     0.8022    0.7919    0.7904     10000

------------------------- 100 -------------------------


100%|██████████| 10000/10000 [02:04<00:00, 80.12it/s]


              precision    recall  f1-score   support

       plane     0.8577    0.7170    0.7810      1000
         car     0.8307    0.8980    0.8630      1000
        bird     0.7722    0.6170    0.6859      1000
         cat     0.5271    0.7110    0.6054      1000
        deer     0.7576    0.4500    0.5646      1000
         dog     0.9727    0.4630    0.6274      1000
        frog     0.5127    0.9460    0.6650      1000
       horse     0.5882    0.7800    0.6707      1000
        ship     0.9187    0.7350    0.8167      1000
       truck     0.9038    0.8080    0.8532      1000

    accuracy                         0.7125     10000
   macro avg     0.7641    0.7125    0.7133     10000
weighted avg     0.7641    0.7125    0.7133     10000

------------------------- 50 -------------------------


100%|██████████| 10000/10000 [02:03<00:00, 81.04it/s]

              precision    recall  f1-score   support

       plane     0.9507    0.2700    0.4206      1000
         car     0.9510    0.7180    0.8182      1000
        bird     0.7983    0.6490    0.7159      1000
         cat     0.5650    0.7780    0.6546      1000
        deer     0.6822    0.7020    0.6920      1000
         dog     0.7507    0.7560    0.7534      1000
        frog     0.8209    0.6280    0.7116      1000
       horse     0.5141    0.8770    0.6482      1000
        ship     0.9066    0.6700    0.7706      1000
       truck     0.6275    0.9570    0.7580      1000

    accuracy                         0.7005     10000
   macro avg     0.7567    0.7005    0.6943     10000
weighted avg     0.7567    0.7005    0.6943     10000

